In [1]:
#import json
import ast

#Find the SCC around a given pivot state
#by finding an intersection of Forwards and Backwards reachability from pivot

def intersect_FB( visitedF, visitedB ):
    result = {} #will be stored as dictionary {rectangle : {entryF facet : list exitF facets}} in the forward direction
    for rkey in visitedF:
        if rkey in visitedB:
            ffacets = visitedF[ rkey ] # entry-> exits forward
            bfacets = visitedB[ rkey ] # exit -> entries from fw point of view
            rresult = { }# facet:list of exits for this given rectangle
            #compare, store the "intersection"
            r_intersectionF = {} #in the form of entry facet F : list of exit facets F
            for fkey in ffacets:
                #if fkey somewhere in list of some bfkey
                for bfkey in ffacets[ fkey ]: #forward [] of exits
                    if bfkey in bfacets: #is exit listed in bacwards?
                        if fkey in bfacets[ bfkey ]: #does exit backwards lead to fw entry?
                            #store
                            if fkey not in r_intersectionF:
                                r_intersectionF[ fkey ] = []
                            r_intersectionF[ fkey ].append( bfkey )
            result[ rkey ] = r_intersectionF #for rectangle r the intersection of fw and bw flow between facets
    return result

def filter_with_entry_exit_pairs( dictionary ):
    result = {}
    for key in dictionary:
        if dictionary[ key ] != {}:
            result[ key ] = dictionary[ key ]
    return result

def write_out_inter( dictionar, i ):
    #write content to the output file
    fo = open("./outBRUinter/output"+str(i)+".txt", "w")
    #fo.write("Intersection of forward and backward reachable states:\n")
    fo.write( str(dictionar) )
    fo.close()

def read_dict_from_file( fb, i ):
    if fb == "f":
        fo = open("./outBRUforward/output"+str(i)+".txt", "r")
    else:
        fo = open("./outBRUbackward/output"+str(i)+".txt", "r")
    data = fo.read()
    fo.close()
    #js = json.loads(data)
    #with_double_quotes = data.replace( "\'", '"' )
    dictionary = ast.literal_eval( data ) #with_double_quotes )
    return dictionary


In [2]:
#visualize the sets, write png images
#need the tresholds and rectangle->geom real
#import
import numpy as np
import seaborn as sns
# Matplotlib for additional customization
from matplotlib import pyplot as plt
%matplotlib inline

import pandas as pd

def r_hash( h ):
    r = [0,0] #default value
    hparts = h.split('|')
    r[0] = int( hparts[0] )
    r[1] = int( hparts[1] )
    return r


In [3]:
#Read forward and backward reachable sets for
#Intersect for rectangles (entry [] the whole rectangle) in init
#write out the results - name them with identifiers "_x_y"
#initial_rectangles = []
identifiers = []
for x in [ 0.5, 1, 1.5, 2, 2.5, 3, 3.5, 4, 4.5, 5, 5.5 ]:
    for y in [ 0.5, 1, 1.5, 2, 2.5, 3, 3.5, 4, 4.5, 5, 5.5  ]:
#        initial_rectangles.append( find_rectangle( [ x, y ] ) )
        identifiers.append( "_" + str( x ) + "_" + str( y )+"B1.7"  )

F_visited_sets = {}
F_union_visited = {}
B_visited_sets = {}
B_union_visited = {}

#for r,i in zip( initial_rectangles, identifiers ):
for i in identifiers:
    visitedF = read_dict_from_file( "f", "Konec"+i )
    F_visited_sets[ i ] = visitedF
    F_union_visited |= visitedF
    
    visitedB = read_dict_from_file( "b", "Konec"+i )
    B_visited_sets[ i ] = visitedB
    B_union_visited |= visitedB
        
    intersect = intersect_FB( visitedF, visitedB )
    write_out_inter( intersect, i )
    intersect = filter_with_entry_exit_pairs( intersect )
    print( intersect )
    #add initial rectangles?

write_out_inter( F_union_visited, "UnionForward"+"B1.7"  )
write_out_inter( B_union_visited, "UnionBackward"+"B1.7"  )


{'8|9': {'10': ['11']}, '8|10': {'10': ['11']}, '9|11': {'01': ['11'], '10': ['00', '11']}, '9|12': {'10': ['00', '11']}, '9|13': {'10': ['00', '11']}, '9|14': {'10': ['00', '11']}, '9|15': {'10': ['00', '11']}, '9|16': {'10': ['00', '11']}, '9|17': {'10': ['00', '11']}, '9|18': {'10': ['00', '11']}, '9|19': {'10': ['00', '11']}, '9|20': {'10': ['11']}, '9|21': {'10': ['11']}, '9|22': {'10': ['11']}, '9|23': {'10': ['11']}, '9|24': {'10': ['11']}, '9|25': {'10': ['11']}, '9|26': {'10': ['11']}, '9|27': {'10': ['11']}, '9|28': {'10': ['11']}, '9|29': {'10': ['11']}, '9|30': {'10': ['11']}, '9|31': {'10': ['11']}, '9|32': {'10': ['11']}, '9|33': {'10': ['11']}, '9|34': {'10': ['11']}, '10|35': {'00': ['11'], '10': ['01', '11']}, '10|36': {'10': ['11']}, '11|37': {'00': ['01'], '10': ['01']}, '12|37': {'00': ['01'], '10': ['01']}, '13|37': {'00': ['01', '10'], '11': ['01']}, '14|37': {'00': ['10', '01'], '11': ['01']}, '14|38': {'00': ['01', '10']}, '15|38': {'00': ['10'], '11': ['01']}, 

{'9|50': {'00': ['01']}, '11|51': {'00': ['01', '10']}, '13|52': {'00': ['10', '01'], '11': ['01']}, '14|53': {'00': ['10'], '11': ['01']}, '15|54': {'00': ['10'], '11': ['01']}, '16|55': {'00': ['10'], '11': ['01']}, '17|56': {'00': ['10'], '11': ['01']}, '18|58': {'00': ['10']}, '19|59': {'00': ['10']}, '20|60': {'00': ['10']}, '21|61': {'00': ['10']}, '22|62': {'00': ['10']}, '23|63': {'00': ['10']}, '24|64': {'00': ['10']}, '25|65': {'00': ['10']}, '26|66': {'00': ['10']}, '27|67': {'00': ['10']}, '28|68': {'00': ['10']}, '29|69': {'00': ['10']}, '30|70': {'00': ['10']}, '31|71': {'00': ['10']}, '32|72': {'00': ['10']}, '33|73': {'00': ['10']}, '34|74': {'00': ['10']}, '35|75': {'00': ['10']}, '36|76': {'00': ['10']}, '37|77': {'00': ['10']}, '38|78': {'00': ['10']}, '39|79': {'00': ['10']}, '40|80': {'00': ['10']}, '41|81': {'00': ['10']}, '42|82': {'00': ['10']}, '43|83': {'00': ['10']}, '44|84': {'00': ['10']}, '45|85': {'00': ['10']}, '46|86': {'00': ['10']}, '47|87': {'00': ['

{'17|25': {'01': ['00', '11'], '10': ['00']}, '18|26': {'01': ['10']}, '18|27': {'11': ['10']}, '19|28': {'00': ['10'], '11': ['10', '01']}, '19|29': {'11': ['01', '10']}, '19|30': {'11': ['01']}, '20|30': {'00': ['10'], '11': ['10', '01']}, '20|31': {'11': ['01', '10']}, '20|32': {'11': ['01']}, '21|32': {'00': ['10'], '11': ['01', '10']}, '21|33': {'11': ['01', '10']}, '22|34': {'00': ['10'], '11': ['10', '01']}, '23|36': {'00': ['10']}, '24|37': {'00': ['10']}, '25|38': {'00': ['10']}, '26|39': {'00': ['10']}, '27|40': {'00': ['10']}, '28|41': {'00': ['10']}, '29|42': {'00': ['10']}, '30|43': {'00': ['10']}, '31|44': {'00': ['10']}, '32|45': {'00': ['10']}, '33|46': {'00': ['10']}, '34|47': {'00': ['10']}, '35|48': {'00': ['10']}, '36|49': {'00': ['10']}, '37|50': {'00': ['10']}, '38|51': {'00': ['10']}, '39|52': {'00': ['10']}, '40|53': {'00': ['10']}, '41|54': {'00': ['10']}, '42|55': {'00': ['10']}, '43|56': {'00': ['10']}, '44|57': {'00': ['10']}, '45|58': {'00': ['10']}, '46|59

{'25|17': {'01': ['00']}, '26|17': {'01': ['00']}, '27|17': {'01': ['00']}, '27|18': {'11': ['00']}, '28|18': {'01': ['10'], '11': ['00']}, '28|19': {'11': ['00', '10']}, '28|20': {'11': ['10']}, '28|21': {'11': ['10']}, '28|22': {'11': ['10']}, '29|25': {'00': ['10']}, '32|30': {'00': ['10']}, '30|27': {'00': ['10']}, '31|28': {'00': ['10']}, '33|31': {'00': ['10']}, '34|32': {'00': ['10']}, '35|33': {'00': ['10']}, '36|34': {'00': ['10']}, '37|35': {'00': ['10']}, '38|36': {'00': ['10']}, '39|37': {'00': ['10']}, '40|38': {'00': ['10']}, '41|39': {'00': ['10']}, '42|40': {'00': ['10']}, '43|41': {'00': ['10']}, '44|42': {'00': ['10']}, '45|43': {'00': ['10']}, '46|44': {'00': ['10']}, '47|45': {'00': ['10']}, '48|46': {'00': ['10']}, '49|47': {'00': ['10']}, '50|48': {'00': ['10']}, '51|49': {'00': ['10']}, '52|50': {'00': ['10']}, '53|51': {'00': ['10']}, '54|52': {'00': ['10']}, '55|53': {'00': ['10']}, '56|54': {'00': ['10']}, '57|55': {'00': ['10']}, '58|56': {'00': ['10']}, '59|

{'33|17': {'11': ['10']}, '33|18': {'11': ['10']}, '33|19': {'11': ['10']}, '33|20': {'11': ['10']}, '34|22': {'00': ['10'], '11': ['10']}, '37|28': {'00': ['10']}, '36|26': {'00': ['10']}, '38|29': {'00': ['10']}, '39|30': {'00': ['10']}, '40|31': {'00': ['10']}, '41|32': {'00': ['10']}, '42|33': {'00': ['10']}, '43|34': {'00': ['10']}, '44|35': {'00': ['10']}, '45|36': {'00': ['10']}, '46|37': {'00': ['10']}, '47|38': {'00': ['10']}, '48|39': {'00': ['10']}, '49|40': {'00': ['10']}, '50|41': {'00': ['10']}, '51|42': {'00': ['10']}, '52|43': {'00': ['10']}, '53|44': {'00': ['10']}, '54|45': {'00': ['10']}, '55|46': {'00': ['10']}, '56|47': {'00': ['10']}, '57|48': {'00': ['10']}, '58|49': {'00': ['10']}, '59|50': {'00': ['10']}, '60|51': {'00': ['10']}, '61|52': {'00': ['10']}, '62|53': {'00': ['10']}, '63|54': {'00': ['10']}, '64|55': {'00': ['10']}, '65|56': {'00': ['10']}, '66|57': {'00': ['10']}, '67|58': {'00': ['10']}, '68|59': {'00': ['10']}, '69|60': {'00': ['10']}, '70|61': {

{'42|9': {'01': ['00']}, '43|9': {'01': ['00', '11']}, '53|18': {'00': ['10']}, '55|21': {'00': ['10']}, '56|22': {'00': ['10']}, '50|10': {'01': ['10']}, '52|15': {'00': ['10'], '11': ['10']}, '54|19': {'00': ['10']}, '53|17': {'00': ['10'], '11': ['10']}, '55|20': {'00': ['10'], '11': ['10']}, '57|23': {'00': ['10']}, '58|24': {'00': ['10']}, '59|25': {'00': ['10']}, '60|26': {'00': ['10']}, '61|27': {'00': ['10']}, '62|28': {'00': ['10']}, '63|29': {'00': ['10']}, '64|30': {'00': ['10']}, '65|31': {'00': ['10']}, '66|32': {'00': ['10']}, '67|33': {'00': ['10']}, '68|34': {'00': ['10']}, '69|35': {'00': ['10']}, '70|36': {'00': ['10']}, '71|37': {'00': ['10']}, '72|38': {'00': ['10']}, '73|39': {'00': ['10']}, '74|40': {'00': ['10']}, '75|41': {'00': ['10']}, '76|42': {'00': ['10']}, '77|43': {'00': ['10']}, '78|44': {'00': ['10']}, '79|45': {'00': ['10']}, '80|46': {'00': ['10']}, '81|47': {'00': ['10']}, '82|48': {'00': ['10']}, '83|49': {'00': ['10']}, '84|50': {'00': ['10']}, '85

{'49|17': {'11': ['10']}, '50|19': {'00': ['10'], '11': ['10']}, '51|21': {'00': ['10']}, '52|23': {'00': ['10']}, '53|24': {'00': ['10']}, '54|25': {'00': ['10']}, '55|26': {'00': ['10']}, '56|27': {'00': ['10']}, '57|28': {'00': ['10']}, '58|29': {'00': ['10']}, '59|30': {'00': ['10']}, '60|31': {'00': ['10']}, '61|32': {'00': ['10']}, '62|33': {'00': ['10']}, '63|34': {'00': ['10']}, '64|35': {'00': ['10']}, '65|36': {'00': ['10']}, '66|37': {'00': ['10']}, '67|38': {'00': ['10']}, '68|39': {'00': ['10']}, '69|40': {'00': ['10']}, '70|41': {'00': ['10']}, '71|42': {'00': ['10']}, '72|43': {'00': ['10']}, '73|44': {'00': ['10']}, '74|45': {'00': ['10']}, '75|46': {'00': ['10']}, '76|47': {'00': ['10']}, '77|48': {'00': ['10']}, '78|49': {'00': ['10']}, '79|50': {'00': ['10']}, '80|51': {'00': ['10']}, '81|52': {'00': ['10']}, '82|53': {'00': ['10']}, '83|54': {'00': ['10']}, '84|55': {'00': ['10']}, '85|56': {'00': ['10']}, '86|57': {'00': ['10']}, '87|58': {'00': ['10']}, '88|59': {

{'59|35': {'00': ['10']}, '60|36': {'00': ['10']}, '61|37': {'00': ['10']}, '62|38': {'00': ['10']}, '63|39': {'00': ['10']}, '64|40': {'00': ['10']}, '65|41': {'00': ['10']}, '66|42': {'00': ['10']}, '67|43': {'00': ['10']}, '68|44': {'00': ['10']}, '69|45': {'00': ['10']}, '70|46': {'00': ['10']}, '71|47': {'00': ['10']}, '72|48': {'00': ['10']}, '73|49': {'00': ['10']}, '74|50': {'00': ['10']}, '75|51': {'00': ['10']}, '76|52': {'00': ['10']}, '77|53': {'00': ['10']}, '78|54': {'00': ['10']}, '79|55': {'00': ['10']}, '80|56': {'00': ['10']}, '81|57': {'00': ['10']}, '82|58': {'00': ['10']}, '83|59': {'00': ['10']}, '84|60': {'00': ['10']}, '85|61': {'00': ['10']}, '86|62': {'00': ['10']}, '87|63': {'00': ['10']}, '88|64': {'00': ['10']}, '89|65': {'00': ['10']}, '90|66': {'00': ['10']}, '91|67': {'00': ['10']}, '92|68': {'00': ['10']}, '93|69': {'00': ['10']}, '94|70': {'00': ['10']}, '95|71': {'00': ['10']}, '96|72': {'00': ['10']}, '97|73': {'00': ['10']}, '98|74': {'00': ['10']},

{'75|68': {'00': ['10']}, '77|70': {'00': ['10']}, '78|71': {'00': ['10']}, '79|72': {'00': ['10']}, '80|73': {'00': ['10']}, '81|74': {'00': ['10']}, '82|75': {'00': ['10']}, '83|76': {'00': ['10']}, '84|77': {'00': ['10']}, '85|78': {'00': ['10']}, '86|79': {'00': ['10']}, '87|80': {'00': ['10']}, '88|81': {'00': ['10']}, '89|82': {'00': ['10']}, '90|83': {'00': ['10']}, '91|84': {'00': ['10']}, '92|85': {'00': ['10']}, '93|86': {'00': ['10']}, '94|87': {'00': ['10']}, '95|88': {'00': ['10']}, '96|89': {'00': ['10']}, '97|90': {'00': ['10']}, '98|91': {'00': ['10']}, '99|92': {'00': ['10']}, '76|69': {'00': ['10']}, '76|68': {'00': ['10'], '11': ['01']}, '77|69': {'00': ['10'], '11': ['01']}, '78|70': {'00': ['10'], '11': ['01']}, '79|71': {'00': ['10'], '11': ['01']}, '80|72': {'00': ['10'], '11': ['01']}, '81|73': {'00': ['10'], '11': ['01']}, '82|74': {'00': ['10'], '11': ['01']}, '83|75': {'00': ['10'], '11': ['01']}, '84|76': {'00': ['10'], '11': ['01']}, '85|77': {'00': ['10'],

In [4]:

#model-specific tresholds
x0min = 0.0
x0max = 6.0
x1min = 0.0
x1max = 6.0

pieces_count = 100

stepx0 = ( x0max - x0min ) / pieces_count
stepx1 = ( x1max - x1min ) / pieces_count

tresholdsX0 = np.arange ( x0min, x0max+(0.3*stepx0), stepx0 )
tresholdsX1 = np.arange ( x1min, x1max+(0.3*stepx1), stepx1 )

tresholds = [ tresholdsX0, tresholdsX1 ]

def visualize_rectangles( rdict, i ):
    list_r = []
    
    for key in rdict:
        r = r_hash( key )
        list_r.append( r )
    
    xlist = []
    ylist = []
    for r in list_r:
        xlist.append( 0.5*( tresholds[0][r[0]] + tresholds[0][r[0]+1]) )
        ylist.append( 0.5*( tresholds[1][r[1]] + tresholds[1][r[1]+1]) )

    ctverecky = pd.DataFrame({
        "x" : xlist,
        "y" : ylist
    })

    wid = 5
    hei = 5
    fig,ax = plt.subplots(1, 1, figsize=(wid, hei))
    sns.scatterplot(x="x", y="y", data=ctverecky, markers="s", s=10)
    ax.set(xlim=(x0min, x0max))
    ax.set(ylim=(x1min, x1max))
    plt.savefig('./outBRUinter/img'+str(i)+'.png', dpi=300)
    plt.clf()
    plt.close()

for i in identifiers:
    visitedF = F_visited_sets[ i ]
    visualize_rectangles( visitedF, "Forw"+i )
    visitedB = B_visited_sets[ i ]
    visualize_rectangles( visitedF, "Backw"+i )
        
    intersect = intersect_FB( visitedF, visitedB )
    write_out_inter( intersect, i )
    intersect = filter_with_entry_exit_pairs( intersect )
    visualize_rectangles( intersect, "Intersection"+i )
    #add initial rectangles?
    
visualize_rectangles( F_union_visited, "UnionForward"+"B1.7" )
visualize_rectangles( B_union_visited, "UnionBackward"+"B1.7" )


In [5]:
#Read forward and backward reachable sets for
#Intersect for rectangles (entry [] the whole rectangle) in init
#write out the results - name them with identifiers "_x_y"
#initial_rectangles = []
identifiers = []
for x in [ 0.5, 1, 1.5, 2, 2.5, 3, 3.5, 4, 4.5, 5, 5.5 ]:
    for y in [ 0.5, 1, 1.5, 2, 2.5, 3, 3.5, 4, 4.5, 5, 5.5  ]:
#        initial_rectangles.append( find_rectangle( [ x, y ] ) )
        identifiers.append( "_" + str( x ) + "_" + str( y )+"B3"  )

F_visited_sets = {}
F_union_visited = {}
B_visited_sets = {}
B_union_visited = {}

#for r,i in zip( initial_rectangles, identifiers ):
for i in identifiers:
    visitedF = read_dict_from_file( "f", "Konec"+i )
    F_visited_sets[ i ] = visitedF
    F_union_visited |= visitedF
    
    visitedB = read_dict_from_file( "b", "Konec"+i )
    B_visited_sets[ i ] = visitedB
    B_union_visited |= visitedB
        
    intersect = intersect_FB( visitedF, visitedB )
    write_out_inter( intersect, i )
    intersect = filter_with_entry_exit_pairs( intersect )
    print( intersect )
    #add initial rectangles?

write_out_inter( F_union_visited, "UnionForward"+"B3"  )
write_out_inter( B_union_visited, "UnionBackward"+"B3"  )


{'8|9': {'10': ['11']}, '9|10': {'01': ['11'], '10': ['00', '11']}, '9|11': {'10': ['00']}, '10|11': {'01': ['11'], '10': ['00', '11']}, '10|12': {'10': ['00', '11']}, '10|13': {'10': ['00']}, '11|13': {'01': ['11'], '10': ['00', '11']}, '11|14': {'10': ['00', '11']}, '11|15': {'10': ['00']}, '12|15': {'01': ['11'], '10': ['00']}, '12|16': {'10': ['00']}, '13|16': {'01': ['11'], '10': ['00']}, '13|17': {'10': ['00']}, '14|17': {'01': ['11'], '10': ['00']}, '14|18': {'10': ['00']}, '15|18': {'01': ['11'], '10': ['00']}, '15|19': {'10': ['00']}, '16|19': {'01': ['11'], '10': ['00']}, '16|20': {'10': ['00']}, '17|20': {'01': ['11'], '10': ['00']}, '17|21': {'10': ['00']}, '18|21': {'01': ['11'], '10': ['00']}, '18|22': {'10': ['00']}, '19|22': {'01': ['11'], '10': ['00']}, '19|23': {'10': ['00']}, '20|23': {'01': ['11'], '10': ['00']}, '20|24': {'10': ['00']}, '21|24': {'01': ['00', '11'], '10': ['00']}, '21|25': {'10': ['00']}, '22|25': {'01': ['00', '11'], '10': ['00']}, '22|26': {'10':

{'8|42': {'10': ['11']}, '8|43': {'10': ['11']}, '9|44': {'01': ['11'], '10': ['00', '11']}, '9|45': {'10': ['11']}, '9|46': {'10': ['11']}, '12|60': {'10': ['11']}, '12|61': {'10': ['11']}, '12|62': {'00': ['11']}, '13|63': {'00': ['10']}, '17|66': {'00': ['10']}, '18|67': {'00': ['10']}, '19|68': {'00': ['10']}, '20|69': {'00': ['10']}, '21|70': {'00': ['10']}, '22|71': {'00': ['10']}, '23|72': {'00': ['10']}, '24|73': {'00': ['10']}, '25|74': {'00': ['10']}, '26|75': {'00': ['10']}, '27|76': {'00': ['10']}, '28|77': {'00': ['10']}, '29|78': {'00': ['10']}, '30|79': {'00': ['10']}, '31|80': {'00': ['10']}, '32|81': {'00': ['10']}, '33|82': {'00': ['10']}, '34|83': {'00': ['10']}, '35|84': {'00': ['10']}, '36|85': {'00': ['10']}, '37|86': {'00': ['10']}, '38|87': {'00': ['10']}, '39|88': {'00': ['10']}, '40|89': {'00': ['10']}, '41|90': {'00': ['10']}, '42|91': {'00': ['10']}, '43|92': {'00': ['10']}, '44|93': {'00': ['10']}, '45|94': {'00': ['10']}, '46|95': {'00': ['10']}, '47|96': 

{'17|25': {'01': ['11'], '10': ['00']}, '17|26': {'10': ['00']}, '18|26': {'01': ['11'], '10': ['00']}, '18|27': {'10': ['00']}, '19|27': {'01': ['11'], '10': ['00']}, '19|28': {'10': ['00']}, '20|28': {'01': ['00', '11'], '10': ['00']}, '20|29': {'10': ['00']}, '21|29': {'01': ['00', '11'], '10': ['00']}, '21|30': {'10': ['00']}, '22|30': {'01': ['00', '11'], '10': ['00']}, '23|31': {'01': ['00'], '10': ['00']}, '24|31': {'01': ['00', '11'], '10': ['00']}, '24|32': {'10': ['00']}, '25|32': {'01': ['00'], '10': ['00']}, '26|32': {'11': ['00']}, '26|33': {'11': ['00']}, '27|33': {'11': ['00', '10']}, '27|34': {'11': ['10']}, '27|35': {'11': ['10']}, '27|36': {'11': ['10']}, '28|36': {'00': ['10'], '11': ['01']}, '28|37': {'11': ['10', '01'], '00': ['10']}, '28|38': {'11': ['01', '10'], '00': ['10']}, '28|39': {'11': ['01', '10'], '00': ['10']}, '29|39': {'00': ['10'], '11': ['01', '10']}, '29|40': {'11': ['01'], '00': ['10']}, '29|41': {'11': ['01', '10'], '00': ['10']}, '30|41': {'00':

{'25|9': {'01': ['11'], '10': ['00']}, '26|10': {'01': ['11'], '10': ['00']}, '27|11': {'01': ['00', '11'], '10': ['00']}, '28|12': {'01': ['00', '11'], '10': ['00']}, '29|13': {'01': ['00', '11'], '10': ['00']}, '30|14': {'01': ['00', '11'], '10': ['00']}, '31|15': {'01': ['00', '11'], '10': ['00']}, '32|16': {'01': ['00'], '10': ['00']}, '33|16': {'01': ['11', '00'], '10': ['00']}, '33|17': {'10': ['00']}, '34|17': {'01': ['00', '11'], '10': ['00']}, '34|18': {'10': ['00']}, '35|18': {'01': ['00', '11'], '10': ['00']}, '35|19': {'10': ['00']}, '36|19': {'01': ['00'], '10': ['00']}, '37|19': {'01': ['00'], '10': ['00']}, '38|19': {'01': ['00', '11'], '10': ['00']}, '38|20': {'10': ['00']}, '39|20': {'01': ['00'], '10': ['00']}, '40|20': {'01': ['00']}, '41|20': {'01': ['00', '10']}, '41|21': {'11': ['00']}, '42|21': {'01': ['10'], '11': ['00']}, '42|22': {'11': ['00', '10']}, '42|23': {'11': ['10']}, '42|24': {'11': ['10']}, '42|25': {'11': ['10']}, '42|26': {'11': ['10']}, '42|27': {

{'24|50': {'11': ['10']}, '25|51': {'00': ['10'], '11': ['01', '10']}, '26|53': {'00': ['10']}, '27|54': {'00': ['10']}, '28|55': {'00': ['10']}, '29|56': {'00': ['10']}, '30|57': {'00': ['10']}, '31|58': {'00': ['10']}, '32|59': {'00': ['10']}, '33|60': {'00': ['10']}, '34|61': {'00': ['10']}, '35|62': {'00': ['10']}, '36|63': {'00': ['10']}, '37|64': {'00': ['10']}, '38|65': {'00': ['10']}, '39|66': {'00': ['10']}, '40|67': {'00': ['10']}, '41|68': {'00': ['10']}, '42|69': {'00': ['10']}, '43|70': {'00': ['10']}, '44|71': {'00': ['10']}, '45|72': {'00': ['10']}, '46|73': {'00': ['10']}, '47|74': {'00': ['10']}, '48|75': {'00': ['10']}, '49|76': {'00': ['10']}, '50|77': {'00': ['10']}, '51|78': {'00': ['10']}, '52|79': {'00': ['10']}, '53|80': {'00': ['10']}, '54|81': {'00': ['10']}, '55|82': {'00': ['10']}, '56|83': {'00': ['10']}, '57|84': {'00': ['10']}, '58|85': {'00': ['10']}, '59|86': {'00': ['10']}, '60|87': {'00': ['10']}, '61|88': {'00': ['10']}, '62|89': {'00': ['10']}, '63|

{'34|24': {'01': ['00']}, '35|25': {'01': ['10'], '11': ['00']}, '35|26': {'11': ['10']}, '35|27': {'11': ['10']}, '35|28': {'11': ['10']}, '35|29': {'11': ['10']}, '36|32': {'00': ['10']}, '39|37': {'00': ['10']}, '37|34': {'00': ['10']}, '38|35': {'00': ['10']}, '40|38': {'00': ['10']}, '41|39': {'00': ['10']}, '42|40': {'00': ['10']}, '43|41': {'00': ['10']}, '44|42': {'00': ['10']}, '45|43': {'00': ['10']}, '46|44': {'00': ['10']}, '47|45': {'00': ['10']}, '48|46': {'00': ['10']}, '49|47': {'00': ['10']}, '50|48': {'00': ['10']}, '51|49': {'00': ['10']}, '52|50': {'00': ['10']}, '53|51': {'00': ['10']}, '54|52': {'00': ['10']}, '55|53': {'00': ['10']}, '56|54': {'00': ['10']}, '57|55': {'00': ['10']}, '58|56': {'00': ['10']}, '59|57': {'00': ['10']}, '60|58': {'00': ['10']}, '61|59': {'00': ['10']}, '62|60': {'00': ['10']}, '63|61': {'00': ['10']}, '64|62': {'00': ['10']}, '65|63': {'00': ['10']}, '66|64': {'00': ['10']}, '67|65': {'00': ['10']}, '68|66': {'00': ['10']}, '69|67': {

{'42|17': {'01': ['00'], '10': ['00']}, '43|17': {'01': ['00'], '10': ['00']}, '44|17': {'01': ['00', '11'], '10': ['00']}, '45|18': {'01': ['00']}, '46|18': {'01': ['00']}, '47|19': {'01': ['10'], '11': ['10']}, '47|20': {'11': ['10']}, '47|21': {'11': ['10']}, '47|22': {'11': ['10']}, '47|23': {'11': ['10']}, '48|24': {'00': ['10'], '11': ['10']}, '48|25': {'11': ['10']}, '49|27': {'00': ['10'], '11': ['10']}, '50|29': {'00': ['10']}, '51|31': {'00': ['10']}, '52|32': {'00': ['10']}, '53|33': {'00': ['10']}, '54|34': {'00': ['10']}, '55|35': {'00': ['10']}, '56|36': {'00': ['10']}, '57|37': {'00': ['10']}, '58|38': {'00': ['10']}, '59|39': {'00': ['10']}, '60|40': {'00': ['10']}, '61|41': {'00': ['10']}, '62|42': {'00': ['10']}, '63|43': {'00': ['10']}, '64|44': {'00': ['10']}, '65|45': {'00': ['10']}, '66|46': {'00': ['10']}, '67|47': {'00': ['10']}, '68|48': {'00': ['10']}, '69|49': {'00': ['10']}, '70|50': {'00': ['10']}, '71|51': {'00': ['10']}, '72|52': {'00': ['10']}, '73|53': 

{'50|16': {'01': ['00']}, '51|16': {'01': ['00']}, '52|16': {'01': ['00']}, '52|17': {'11': ['00']}, '53|17': {'01': ['10'], '11': ['10']}, '53|18': {'11': ['10', '00']}, '53|19': {'11': ['10']}, '53|20': {'11': ['10']}, '53|21': {'11': ['01', '10']}, '53|22': {'11': ['01']}, '54|22': {'00': ['10'], '11': ['10']}, '54|23': {'11': ['10', '01']}, '54|24': {'11': ['01', '10']}, '54|25': {'11': ['01']}, '55|25': {'00': ['10'], '11': ['01', '10']}, '55|26': {'11': ['01', '10']}, '56|27': {'00': ['10'], '11': ['01', '10']}, '56|28': {'11': ['01', '10']}, '57|29': {'00': ['10'], '11': ['10', '01']}, '58|31': {'00': ['10']}, '59|32': {'00': ['10']}, '60|33': {'00': ['10']}, '61|34': {'00': ['10']}, '62|35': {'00': ['10']}, '63|36': {'00': ['10']}, '64|37': {'00': ['10']}, '65|38': {'00': ['10']}, '66|39': {'00': ['10']}, '67|40': {'00': ['10']}, '68|41': {'00': ['10']}, '69|42': {'00': ['10']}, '70|43': {'00': ['10']}, '71|44': {'00': ['10']}, '72|45': {'00': ['10']}, '73|46': {'00': ['10']}, 

{'59|43': {'00': ['10']}, '60|44': {'00': ['10']}, '61|45': {'00': ['10']}, '62|46': {'00': ['10']}, '63|47': {'00': ['10']}, '64|48': {'00': ['10']}, '65|49': {'00': ['10']}, '66|50': {'00': ['10']}, '67|51': {'00': ['10']}, '68|52': {'00': ['10']}, '69|53': {'00': ['10']}, '70|54': {'00': ['10']}, '71|55': {'00': ['10']}, '72|56': {'00': ['10']}, '73|57': {'00': ['10']}, '74|58': {'00': ['10']}, '75|59': {'00': ['10']}, '76|60': {'00': ['10']}, '77|61': {'00': ['10']}, '78|62': {'00': ['10']}, '79|63': {'00': ['10']}, '80|64': {'00': ['10']}, '81|65': {'00': ['10']}, '82|66': {'00': ['10']}, '83|67': {'00': ['10']}, '84|68': {'00': ['10']}, '85|69': {'00': ['10']}, '86|70': {'00': ['10']}, '87|71': {'00': ['10']}, '88|72': {'00': ['10']}, '89|73': {'00': ['10']}, '90|74': {'00': ['10']}, '91|75': {'00': ['10']}, '92|76': {'00': ['10']}, '93|77': {'00': ['10']}, '94|78': {'00': ['10']}, '95|79': {'00': ['10']}, '96|80': {'00': ['10']}, '97|81': {'00': ['10']}, '98|82': {'00': ['10']},

{'75|85': {'00': ['10']}, '77|87': {'00': ['10']}, '78|88': {'00': ['10']}, '79|89': {'00': ['10']}, '80|90': {'00': ['10']}, '81|91': {'00': ['10']}, '82|92': {'00': ['10']}, '83|93': {'00': ['10']}, '84|94': {'00': ['10']}, '85|95': {'00': ['10']}, '86|96': {'00': ['10']}, '87|97': {'00': ['10']}, '88|98': {'00': ['10']}, '89|99': {'00': ['10']}, '76|86': {'00': ['10']}, '75|84': {'00': ['10'], '11': ['01']}, '76|85': {'00': ['10'], '11': ['01']}, '77|86': {'00': ['10'], '11': ['01']}, '78|87': {'00': ['10'], '11': ['01']}, '79|88': {'00': ['10'], '11': ['01']}, '80|89': {'00': ['10'], '11': ['01']}, '81|90': {'00': ['10'], '11': ['01']}, '82|91': {'00': ['10'], '11': ['01']}, '83|92': {'00': ['10'], '11': ['01']}, '84|93': {'00': ['10'], '11': ['01']}, '85|94': {'00': ['10'], '11': ['01']}, '86|95': {'00': ['10'], '11': ['01']}, '87|96': {'00': ['10'], '11': ['01']}, '88|97': {'00': ['10'], '11': ['01']}, '89|98': {'00': ['10'], '11': ['01']}, '90|99': {'00': ['10'], '11': ['01']}}


In [6]:

#model-specific tresholds
x0min = 0.0
x0max = 6.0
x1min = 0.0
x1max = 6.0

pieces_count = 100

stepx0 = ( x0max - x0min ) / pieces_count
stepx1 = ( x1max - x1min ) / pieces_count

tresholdsX0 = np.arange ( x0min, x0max+(0.3*stepx0), stepx0 )
tresholdsX1 = np.arange ( x1min, x1max+(0.3*stepx1), stepx1 )

tresholds = [ tresholdsX0, tresholdsX1 ]

def visualize_rectangles( rdict, i ):
    list_r = []
    
    for key in rdict:
        r = r_hash( key )
        list_r.append( r )
    
    xlist = []
    ylist = []
    for r in list_r:
        xlist.append( 0.5*( tresholds[0][r[0]] + tresholds[0][r[0]+1]) )
        ylist.append( 0.5*( tresholds[1][r[1]] + tresholds[1][r[1]+1]) )

    ctverecky = pd.DataFrame({
        "x" : xlist,
        "y" : ylist
    })

    wid = 5
    hei = 5
    fig,ax = plt.subplots(1, 1, figsize=(wid, hei))
    sns.scatterplot(x="x", y="y", data=ctverecky, markers="s", s=10)
    ax.set(xlim=(x0min, x0max))
    ax.set(ylim=(x1min, x1max))
    plt.savefig('./outBRUinter/img'+str(i)+'.png', dpi=300)
    plt.clf()
    plt.close()

for i in identifiers:
    visitedF = F_visited_sets[ i ]
    visualize_rectangles( visitedF, "Forw"+i )
    visitedB = B_visited_sets[ i ]
    visualize_rectangles( visitedF, "Backw"+i )
        
    intersect = intersect_FB( visitedF, visitedB )
    write_out_inter( intersect, i )
    intersect = filter_with_entry_exit_pairs( intersect )
    visualize_rectangles( intersect, "Intersection"+i )
    #add initial rectangles?
    
visualize_rectangles( F_union_visited, "UnionForward"+"B3" )
visualize_rectangles( B_union_visited, "UnionBackward"+"B3" )
